# Forensics - Read Data from Log Analytics

__Notebook Version:__ 1.0<br>
__Python Version:__ Python 3.6 (including Python 3.6 - AzureML)<br>
__Required Packages:__ Kqlmagic 0.1.90<br>
__Platforms Supported:__<br>
    -  Azure Notebooks Free Compute
    -  Azure Notebooks DSVM
__Data Source Required:__<br>
    -  Log Analytics - SecurityEvent
    
### Description
The notebook provides you basic knowledge to use load data from Azure Log Analytics.

<font color=red>When you switch between Azure Notebooks Free Compute and Data Science Virtual Machine (DSVM), you may need to select Python version: please select Python 3.6 for Free Compute, and Python 3.6 - AzureML for DSVM.</font>

## Prerequisite check

In [ ]:
# only run once, current version 0.1.2
!pip install --upgrade Sentinel-Utilities

In [ ]:
import SentinelUtils
# checking Python version
check = SentinelUtils.version_management.ModuleVersionCheck()
py_check = check.validate_python('3.6.0')
if py_check.requirement_met == False:
    print('Please select Python 3.6 or Python 3.6 - AzureML at the upper right corner')
else:
    print('Please continue')

In [ ]:
# checking required packages
mods_check = check.validate_installed_modules(['Kqlmagic>=0.1.90'])
for mod_info in mods_check:
    if mod_info.requirement_met == False:
        print('Please install {} {} at the following cell.'.format(mod_info.name, mod_info.required_version))

In [ ]:
# Please install required packages based on the check at last cellp
#!pip install Kqlmagic --upgrade

## Table of Contents

1. Retrieve Log Analytics information 
2. Log into Log Analytics
3. Check Data Availability and Query Data
4. Retrieve KQL from Azure Sentinel GitHub
5. Get Azure Sentinel HuntingBookmarks

## 1. Retrieve Log Analytics Information

In [ ]:
path = %env PATH
dsvm = False
if '/dsvm/' in path:
    dsvm = True
    
if dsvm == False:
    # Run this if you are using Free Compute
    tenant_id = SentinelUtils.config_reader.ConfigReader.read_config_values("config.json")[0]
    subscription_id = SentinelUtils.config_reader.ConfigReader.read_config_values("config.json")[1]
    resource_group = SentinelUtils.config_reader.ConfigReader.read_config_values("config.json")[2]
    workspace_id = SentinelUtils.config_reader.ConfigReader.read_config_values("config.json")[3]
    workspace_name = SentinelUtils.config_reader.ConfigReader.read_config_values("config.json")[4]
    print('Your Log Analytic Workspace: {}'.format(workspace_name))
else:
    # Run this if you are using DSVM.  You need to copy the values from config.json, if the file has no value, then you need to go to Log Analytics Portal to get the information.
    tenant_id = input('tenant_id:')
    subscription_id = input('subscription_id:')
    resource_group = input('resource_group:')
    workspace_id = input('workspace_id:')
    workspace_name = input('workspace_name:')

## 2. Log into Log Analytics

In [ ]:
# You must run this cell to log into Log Analytics to continue
# Make sure you have 0.1.90 or above, if not, run Kqlmagic installation again
%reload_ext Kqlmagic
%kql loganalytics://code;workspace=workspace_id;tenant=tenant_id;alias="SentinelDB"

## 3. Check Data Availability and Query Data

1. Tables with records

In [ ]:
%kql search * | summarize count() by Type

2. Query Security Event

In [ ]:
%kql SecurityEvent | take 5 | project Account, Computer
df = _kql_raw_result_.to_dataframe()
df

## 4. Retrieve KQL from Azure Sentinel GitHub

In [ ]:
# Load query from GitHub
import requests
import json
class GitHubHelper(object):
    def __init__(self):
        github_url = "https://raw.githubusercontent.com/Azure/Azure-Sentinel/master/Hunting%20Queries/DeployedQueries.json"
        response = requests.get(github_url)
        response.encoding = response.apparent_encoding
        self.queries = json.loads(response.text)
    
    def get_queries(self):
        if self.queries != None:
            try:
                return [query['name'] for query in self.queries]
            except Exception as e:
                print(e)
                return None

    def get_query(self, query_name):
        q = None
        if self.queries != None:
            try:
                entry = [query for query in self.queries if query['name'] == query_name]
                if entry is not None:
                    q = entry[0]['query']
            except Exception as e:
                print(e)
            finally:
                return q

In [ ]:
# select a query to continue
github_help = GitHubHelper()
query_list = github_help.get_queries()
import ipywidgets as widgets
from IPython.display import display
selected_query = widgets.Dropdown(options=query_list, value=query_list[0],description='query list:')
display(selected_query)

In [ ]:
# Run query
query_text = github_help.get_query(selected_query.value)
%kql {query_text}

## 5. Get Azure Sentinel HuntingBookmarks

In [ ]:
%kql HuntingBookmark | take 5 
df = _kql_raw_result_.to_dataframe()
display(df)